#### Import Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from data_utils import *
from tqdm import tnrange, tqdm_notebook

#### Load Data

In [ ]:
train_valid_ligids  = np.load('../data/PHARM_TRAIN_X.npy')
train_valid_smiles = np.load('../data/PHARM_TRAIN_SMILES.npy')
train_valid_scores = np.load('../data/Y_train.npy')

print('train_valid_ligids shape: {}'.format(train_valid_ligids.shape))
print('train_valid_smiles shape: {}'.format(train_valid_smiles.shape))
print('train_valid_scores shape: {}'.format(train_valid_scores.shape))

#### Preprocess Data

In [ ]:
# Dimentionality reduction
train_valid_ligids = reduce_dimensions(train_valid_ligids, lower_thresh=1000, upper_thresh=None)
train_valid_smiles = reduce_dimensions(train_valid_smiles, lower_thresh=1000, upper_thresh=44000)
print('reduced train_valid_ligids shape: {}'.format(train_valid_ligids.shape))
print('reduced train_valid_smiles shape: {}'.format(train_valid_smiles.shape))

# Balance Classes
map_from = [0,1,2,3,4,5,6,7,8,9]
map_to =   [0,0,0,0,1,2,3,4,4,4]
train_valid_scores = remap_scores(train_valid_scores, map_from, map_to)

# Train Validation Split
train_data, validation_data = train_validation_split(train_valid_ligids,
                                                     train_valid_smiles,
                                                     train_valid_scores,
                                                     num_val_lig=3046, 
                                                     num_val_smi=10581,
                                                     shuffle=False)
del train_valid_ligids, train_valid_smiles, train_valid_scores

# # Use Fraction of Dataset
# train_data = use_fraction_dataset(train_data, percent=.05)
# validation_data = use_fraction_dataset(validation_data, percent=.05)

#### Define Model

In [ ]:
# Other Parameters
use_regression = True
train_summaries = []
validation_summaries = []

# Hyper Parameters
LEARNING_RATE = .0001
LAMBDA = .000
DROPOUT = .5
NUM_EPOCHS = 5
TRAINING_BATCH_SIZE = 1000
USE_PERCENT_TRAIN_DATA = .05
USE_PERCENT_VALID_DATA = .05
VALIDATION_BATCH_SIZE = 10000

# Model
L1_UNITS = 20
if use_regression:
    NUM_OUTPUTS = 1
else:
    NUM_OUTPUTS = 5

tf.reset_default_graph()

relu = tf.nn.relu
elu = tf.nn.elu
xavier_init = tf.contrib.layers.xavier_initializer()
zero_init = tf.zeros_initializer()
l2_reg = tf.contrib.layers.l2_regularizer(scale=LAMBDA)

with tf.name_scope('inputs') as scope:
    X = tf.placeholder(shape=(None, 126), dtype=tf.float32, name='ligids_smiles')
    Y = tf.placeholder(shape=(None), dtype=tf.float32, name='score')    
    
    
with tf.name_scope('hidden_layers') as scope:
    layer1 = tf.layers.dense(inputs=X,
                             units=L1_UNITS, 
                             activation=relu,
                             kernel_initializer=xavier_init,
                             bias_initializer=zero_init,
                             kernel_regularizer=l2_reg,
                             bias_regularizer=l2_reg,
                             name='layer1')
    

with tf.name_scope('predicted_score') as scope:
    logits = tf.layers.dense(inputs=layer1,
                             units=NUM_OUTPUTS,
                             name='logits')
    if use_regression: # Regression
        with tf.get_default_graph().gradient_override_map({"Floor": "Identity"}):
            pred_score = tf.floor(logits, name='pred_score')
    else: # Classification  
        pred_score = tf.nn.softmax(logits=logits, axis=1)
        pred_score = tf.argmax(pred_score, axis=1, name='pred_score')
        
        
with tf.name_scope('loss') as scope:
    if use_regression: # Regression
        model_loss = tf.losses.absolute_difference(Y,pred_score) # MAE
#         model_loss = tf.losses.mean_squared_error(labels=Y, predictions=pred_score) # MSE
    else: # Classification
        model_loss = tf.reduce_sum(tf.losses.sparse_softmax_cross_entropy(logits=logits, labels=Y))
    reg_loss = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    loss = model_loss+reg_loss
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss)

with tf.name_scope('train_summaries') as scope:
    train_summaries.append(tf.summary.histogram('ligids_smiles', X))
    train_summaries.append(tf.summary.histogram('score', Y))
    train_summaries.append(tf.summary.histogram('layer1', layer1))
    train_summaries.append(tf.summary.histogram('pred_score', pred_score))
    train_summaries.append(tf.summary.scalar('model_loss', model_loss))
    train_summaries.append(tf.summary.scalar('reg_loss', reg_loss))
    train_summaries.append(tf.summary.scalar('loss', loss))

with tf.name_scope('validation_summaries') as scope:
    validation_summaries.append(tf.summary.histogram('ligids_smiles', X))
    validation_summaries.append(tf.summary.histogram('score', Y))
    validation_summaries.append(tf.summary.histogram('layer1', layer1))
    validation_summaries.append(tf.summary.histogram('pred_score', pred_score))
    validation_summaries.append(tf.summary.scalar('model_loss', model_loss))
    validation_summaries.append(tf.summary.scalar('reg_loss', reg_loss))
    validation_summaries.append(tf.summary.scalar('loss', loss))

#### Run Model

In [ ]:
# Misc. constants
num_training_batches = int(train_data.num_scores/TRAINING_BATCH_SIZE*USE_PERCENT_TRAIN_DATA)
num_validation_batches = int(validation_data.num_scores/VALIDATION_BATCH_SIZE*USE_PERCENT_VALID_DATA)
TB_PERIOD_TRAIN = 100
TB_PERIOD_VALID = 1

# Saver - Prediction Tensors
tf.get_collection_ref('pred_ops').clear()
tf.get_collection('pred_ops')
tf.add_to_collection('pred_ops', X)
tf.add_to_collection('pred_ops', Y)
tf.add_to_collection('pred_ops', pred_score)
# Saver - Training Tensors
tf.get_collection_ref('train_ops').clear()
tf.get_collection('train_ops')
tf.add_to_collection('train_ops', X)
tf.add_to_collection('train_ops', Y)
tf.add_to_collection('train_ops', loss)
tf.add_to_collection('train_ops', train_op)
saver = tf.train.Saver(max_to_keep=1000)

# Tensorboard - Graph
time_now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
tensorboard_logdir = '../tf_log/run-{}'.format(time_now)
print('tensorboard log_dir: {}'.format(tensorboard_logdir))
writer = tf.summary.FileWriter(tensorboard_logdir)
writer.add_graph(tf.get_default_graph())

# Start Session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in tqdm_notebook(range(NUM_EPOCHS),desc='Epoch'):       
        # Train Model
        train_data.reset()
#         train_data.shuffle()
        for step in tqdm_notebook(range(num_training_batches), desc='Train', leave=False):
            ligids_batch, smiles_batch, scores_batch = train_data.next_batch(TRAINING_BATCH_SIZE)
            lig_smi_batch = np.concatenate((ligids_batch,smiles_batch), axis=1)
            _, train_loss = sess.run([train_op, loss], feed_dict={X:lig_smi_batch, Y:scores_batch})
            # Tensorboard - Train
            if step%TB_PERIOD_TRAIN == 0:
                s = [sess.run(summary, feed_dict={X:lig_smi_batch, Y:scores_batch})\
                     for summary in train_summaries]
                global_train_step = step+epoch*num_training_batches
                [writer.add_summary(summary, global_train_step) for summary in s]
                
        # Validation
        mae_value = []
        validation_data.reset()
        for step in tqdm_notebook(range(num_validation_batches), desc='Validation', leave=False): 
            ligids_batch, smiles_batch, scores_batch = validation_data.next_batch(VALIDATION_BATCH_SIZE)
            lig_smi_batch = np.concatenate((ligids_batch, smiles_batch), axis=1)
            mae_batch = sess.run(model_loss, feed_dict={X:lig_smi_batch, Y:scores_batch})
            mae_value.append(mae_batch)
            # Tensorboard - Validation
            if step%TB_PERIOD_VALID == 0:
                s = [sess.run(summary, feed_dict={X:lig_smi_batch, Y:scores_batch})\
                     for summary in validation_summaries]
                global_valid_step = step+epoch*num_validation_batches
                [writer.add_summary(summary, global_valid_step) for summary in s]
        mae_value = sum(mae_value)/len(mae_value)
        
        # Save Model w/ name: mse{mean_abs_err}_e{epoch number}
        saver_filename = 'mae{}_e{}'.format(mae_value, epoch+1)        
        saver.save(sess, '../models/{}/{}'.format(time_now, saver_filename))